In [87]:
with open('input.txt', 'r') as file:
    initial_map = file.read().splitlines()

In [2]:
def get_pdir(_map):
    for y, row in enumerate(_map):
        for x, column in enumerate(row):
            if column in ['<', '>', '^', 'v']:
                return x, y, column
    return None

In [3]:
def move_forward(px, py, cd):
    if cd == '<':
        return px-1, py
    elif cd == '>':
        return px+1, py
    elif cd == '^':
        return px, py-1
    elif cd == 'v':
        return px, py+1

In [4]:
def turn_right(cd):
    if cd == '<':
        return '^'
    elif cd ==  '^':
        return '>'
    elif cd ==  '>':
        return 'v'
    elif cd == 'v':
        return '<'

In [5]:
def get_cell_facing(px, py, cd):
    if cd == '<':
        return px-1, py
    elif cd ==  '>':
        return px+1, py
    elif cd ==  '^':
        return px, py-1
    elif cd == 'v':
        return px, py+1

In [6]:
def has_obstacle(px, py, cd, _map):
    cx, cy = get_cell_facing(px, py, cd)
    return '#' in _map[cy][cx]

In [7]:
def set_visited(px, py, cd, _map):
    _map[py] = _map[py][:px] + [_map[py][px], cd] + _map[py][px+1:]

In [8]:
def add_obstacle(ox, oy, _map, icon='#'):
    _map[oy] = _map[oy][:ox] + [icon] + _map[oy][ox+1:]

In [82]:
def has_finished(px, py, cd, _map):
    return (
        (px <= 0 and cd == '<') or
        (px >= len(_map[0]) - 1 and cd == '>') or
        (py <= 0 and cd == '^') or
        (py >= len(_map) - 1 and cd == 'v')
    )

In [22]:
def on_prev_route(px, py, cd, _map):
    return cd in _map[py][px]

In [11]:
def out_of_bounds(px, py, _map):
    height = len(_map)
    width = len(_map[0])
    return px < 0 or py < 0 or px > width-1 or py > height-1

In [12]:
def add_current_pos(px, py, cd, _map):
    current_cell = _map[py][px]
    if '.' in current_cell:
        _map[py][px] = cd
    else:
        _map[py][px] += cd

In [13]:
import concurrent.futures
import copy

In [75]:
def check_for_loop(px, py, cd, ox, oy, _map_) -> bool:
    if out_of_bounds(ox, oy, _map_):
        return False
    
    _map = copy.deepcopy(_map_)
    add_obstacle(ox, oy, _map)

    visited = set()
    
    while True:
            
        if has_obstacle(px, py, cd, _map):
            cd = turn_right(cd)
        else:
            px, py = move_forward(px, py, cd)

        if has_finished(px, py, cd, _map):
            return False
        
        state = (px, py, cd)
        
        if state in visited and not runs_to_end(px, py, cd, _map):
            return True
        
        visited.add(state)

In [76]:
def runs_to_end(px, py, cd, _map):
    while True:
        px, py = move_forward(px, py, cd)
        if has_finished(px, py, cd, _map):
            return True
        if has_obstacle(px, py, cd, _map):
            return False

In [77]:
def simulate(px, py, cd, _map):
    
    obstacle_map = copy.deepcopy(_map)
    
    with concurrent.futures.ThreadPoolExecutor() as executor:
        while True:
            if has_obstacle(px, py, cd, _map):
                cd = turn_right(cd)
            else:
                px, py = move_forward(px, py, cd)
            
            if has_finished(px, py, cd, _map):
                break
            
            ox, oy = get_cell_facing(px, py, cd)
        
            if not out_of_bounds(ox, oy, _map):
                future = executor.submit(check_for_loop, px, py, cd, ox, oy, _map)
                
                if future.result():
                    add_obstacle(ox, oy, obstacle_map, icon='O')
        
            add_current_pos(px, py, cd, _map)

    return _map, obstacle_map

In [88]:
px, py, cd = get_pdir(initial_map)
char_map = [[[char] for char in row] for row in initial_map]
_map, obstacle_map = simulate(px, py, cd, char_map)

In [89]:
for row in _map:
    print("".join(f"{''.join(cell):<2}" for cell in row))

. . . . . . . # . . . . . . . . . # . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . # . . . . . . # . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . # . . # . . . . . . . . . . . . . . . # . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . # # . . . . . . . . . # . . . . . . . . . . # . . . . . # # . . . # . . . # . . . . . # . . . . # . . . . . . . . . . . . . . . . . . # . . . . . . . . . . . . . 
# . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . # . . # . . . . . . . . . . . . . # . . . . . . . . . . # . # # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [90]:
for row in obstacle_map:
    print("".join(f"{''.join(cell):<2}" for cell in row))

. . . . . . . # . . . . . . . . . # . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . # . . . . . . # . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . # . . # . . . . . . . . . . . . . . . # . . . . . . . . . . . . 
. . . . . . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
. . . . . . . . . . . . . . . # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . # # . . . . . . . . . # . . . . . . . . . . # . . . . . # # . . . # . . . # . . . . . # . . . . # . . . . . . . . . . . . . . . . . . # . . . . . . . . . . . . . 
# . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . # . . # . . . . . . . . . . . . . # . . . . . . . . . . # . # # . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . .

In [91]:
sum(row.count('O') for row in obstacle_map)

1972